# Tutorial - Integrating RAPIDS with BERTopic for Topic Modeling 
(last updated 08-19-2022)

In this tutorial, we will use GPU to gauge the perfromance speedup that can be achieved by integrating RAPIDS with BERTopic. 

<br>

<img src="https://cdn-images-1.medium.com/max/803/0*jRTe8f9xmQN3SCCX" width="40%">

# Installing RAPIDS

- First, you'll need to ensure GPUs are enabled for the notebook
- Navigate to [RAPIDS](https://rapids.ai/start.html) to install latest CUDA driver and RAPIDS container
- For this tutorial, I installed RAPIDS docker image for Python 3.9 and Ubuntu 20.04

# Installing BERTopic and other necessary packages

### Installing BERTopic

We need to ensure that HDBSCAN package version is compatible with the environment to seamlessly install BERTopic.

In [1]:
!pip uninstall torch torchvision torchaudio -y

In [2]:
# Downloading a speicific version of torch which is compatible with BERTopic
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 3.0 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 123.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 2.5 MB/s eta 0:00:00a 0:00:010m


In [3]:
# Can be avoided if you want to pip install
!conda install -c conda-forge hdbscan -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda/envs/rapids

  added / updated specs:
    - hdbscan


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.6.15  |       ha878542_0         149 KB  conda-forge
    certifi-2022.6.15          |   py39hf3d152e_0         155 KB  conda-forge
    hdbscan-0.8.28             |   py39hce5d2b2_1         706 KB  conda-forge
    openssl-1.1.1q             |       h166bdaf_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  hdbscan            conda-forge/linux-64::hdbscan-0.8.28-p

In [4]:
!conda update -n base conda -y

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::brotlipy==0.7.0=py39h3811e60_1003
  - conda-forge/linux-64::certifi==2022.5.18.1=py39hf3d152e_0
  - conda-forge/linux-64::cffi==1.15.0=py39h4bc2ebd_0
  - conda-forge/noarch::charset-normalizer==2.0.12=pyhd8ed1ab_0
  - conda-forge/noarch::colorama==0.4.4=pyh9f0ad1d_0
  - conda-forge/linux-64::conda==4.12.0=py39hf3d152e_0
  - conda-forge/linux-64::conda-package-handling==1.8.0=py39hb9d737c_0
  - conda-forge/linux-64::cryptography==36.0.2=py39hd97740a_0
  - conda-forge/noarch::idna==3.3=pyhd8ed1ab_0
  - conda-forge/noarch::pip==22.0.4=pyhd8ed1ab_0
  - conda-forge/linux-64::pycosat==0.6.3=py39h3811e60_1009
  - conda-forge/noarch::pycparser==2.21=pyhd8ed1ab_0
  - conda-forge/noarch::pyopenssl==22.0.0=pyhd8ed1ab_0
  - conda-forge/linux-64::pysocks==1.7.1=py39hf3d152e_4
  - conda-forge/linux-64::python==3.9.10=h85951f

In [5]:
# Can be avoided if HDBSCAN is installed from Conda
!pip install --upgrade pip hdbscan wheel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 145.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.1.2
    Uninstalling pip-22.1.2:
      Successfully uninstalled pip-22.1.2


In [6]:
!pip install bertopic 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 178.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 80.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.1/630.1 kB 219.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 226.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 258.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 258.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 253.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.1/765.1 kB 229.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125940 sha256=39a50fd268f9ad01c09c0b00c26

In [7]:
#Needed to access folders
# !pip install boto3

In [8]:
#Needed to access Amazon S3 buckets
# !pip install awswrangler

### Installing package needed for downloading Wikidata Corpus

In [7]:
#Needed to access Wikipedia corpus
!pip install apache_beam mwparserfromhell

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 200.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 126.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 114.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 213.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 82.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 234.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 120.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for crcmod: filename=crcmod-1.7-py3-none-any.whl size=18848 sha256=fe70632bbfa39dacfd2d0d2e8d28c233001fd0fdf27400323d727c4ef

In [8]:
#Needed to access Wikipedia corpus
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 107.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 150.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 87.9 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.1.1
    Uninstalling dill-0.3.1.1:
      Successfully uninstalled dill-0.3.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.40.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.5.1 which is incompatible.


### Restart the Notebook
After installing BERTopic, we should restart the notebook to correctly use the updated packages.

Go to Menu -> Runtime → Restart Runtime

# Data Loading and Preprocessing

In [1]:
#Importing necessary packages
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import torch
import rmm
import os
# import boto3
import sys
import pandas as pd
import csv
import io
from bertopic import BERTopic
from hdbscan import HDBSCAN
from umap import UMAP
import numpy as np
import time
pd.set_option('max_colwidth', -1)
os.environ["TOKENIZERS_PARALLELISM"] = "true"
rmm.reinitialize(pool_allocator=True,initial_pool_size=5e+9)

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Check if GPU is working
import torch 
torch.tensor([1.0,2.0]).cuda()

tensor([1., 2.], device='cuda:0')

### Data Loading

We are using 4 publicly available datasets for demonstration
- Newsgroups dataset containing roughly 18000 newsgroups posts.
- Contract Understanding Atticus Dataset (CUAD) dataset corpus consisting of more than 13,000 labels in 510 commercial legal contracts. 
- AN4 dataset consisting of spectrogram information that is transformed using Soumith's audio library.
- Wikidata corpus built from the Wikipedia dump (https://dumps.wikimedia.org/) contains the contents of multiple Wikipedia articles.

To add additional datasets,
- Put datasets in data folder. Code is generalized to handle additional datasets that are put in data folder.
- But import datasets need to manually added in a list 'bucket_list' that keeps track of all the datasets to be used.

In [7]:
#Importing custom datasets for the lab
#All the datasets names are stored in a list 'bucket_list' for ease of tracking 
bucket_list = []

#Importing news dataset
from sklearn.datasets import fetch_20newsgroups
docs = fetch_20newsgroups(subset='all')['data']
bucket_list.append('docs')

In [8]:
#Code will pickup files from data folder and add it to bucket_list
data_folder = "./data/"
list_files = os.listdir(data_folder)
for i in list_files:
    if i.find(".md")==-1:
        bucket_list.append(data_folder+i)

bucket_list

#Following lines of code help with picking up data files from S3 bucket
#Loading data from S3 bucket
# s3_client = boto3.client('s3')
# s3_bucket_name = 'nvidia-rapids'
# s3 = boto3.resource('s3')
# my_bucket=s3.Bucket(s3_bucket_name)

# for file in my_bucket.objects.filter():
#     file_name=file.key
#     if file_name!="data/" and file_name.find("data/")!=-1:
#         bucket_list.append(file.key)
# bucket_list

['docs', './data/CUADv1.json', './data/an4_sphere.tar.gz']

In [10]:
#Importing Wikidata corpus
from datasets import load_dataset
data_wiki = pd.DataFrame(load_dataset("wikipedia", "20220301.en"))
bucket_list.append('data_wiki')

bucket_list

OSError: Not enough disk space. Needed: 38.07 GiB (download: 19.18 GiB, generated: 18.88 GiB, post-processed: Unknown size)

In [12]:
#Code to read the data files
data_wiki_rows = 500000  #As Wikipedia data is huge, to ensure a lower runtime, we can update this variable to pickup these many number of rows 
for i in range(len(bucket_list)):
    if bucket_list[i].find(".json")!=-1:
        globals()["data_" + str(i)] = pd.read_json(bucket_list[i], orient='columns')
    elif bucket_list[i].find(".tar.gz")!=-1:
        globals()["data_" + str(i)] = pd.read_csv(bucket_list[i], compression='gzip', header=None, encoding="cp437", delimiter =';', engine='python', on_bad_lines='skip')
        # globals()["data_" + str(i)] = pd.read_csv('s3://'+s3_bucket_name+'/'+bucket_list[i], compression='gzip', header=0, sep=' ', on_bad_lines='skip')
    elif bucket_list[i].find(".pkl")!=-1:
        globals()["data_" + str(i)] = pd.read_pickle(bucket_list[i])
    elif bucket_list[i]=='docs':
        globals()["data_" + str(i)] = docs
    elif bucket_list[i]=='data_wiki':
        globals()["data_" + str(i)] = data_wiki.head(data_wiki_rows)
    else:
        globals()["data_" + str(i)] = pd.read_csv(bucket_list[i])

#Following lines of code help with reading data files from S3 bucket
# for i in range(len(bucket_list)):
#     if bucket_list[i].find(".json")!=-1:
#         globals()["data_" + str(i)] = pd.read_json('s3://'+s3_bucket_name+'/'+bucket_list[i], orient='columns')
#     elif bucket_list[i].find(".tar.gz")!=-1:
#         globals()["data_" + str(i)] = pd.read_csv('s3://'+s3_bucket_name+'/'+bucket_list[i], compression='gzip', header=None, encoding="cp437", delimiter =';', engine='python', on_bad_lines='skip')
#         # globals()["data_" + str(i)] = pd.read_csv('s3://'+s3_bucket_name+'/'+bucket_list[i], compression='gzip', header=0, sep=' ', on_bad_lines='skip')
#     elif bucket_list[i].find(".pkl")!=-1:
#         globals()["data_" + str(i)] = pd.read_pickle('s3://'+s3_bucket_name+'/'+bucket_list[i])
#     elif bucket_list[i]=='docs':
#         globals()["data_" + str(i)] = docs
#     elif bucket_list[i]=='data_wiki':
#         globals()["data_" + str(i)] = data_wiki
#     else:
#         globals()["data_" + str(i)] = pd.read_csv('s3://'+s3_bucket_name+'/'+bucket_list[i])

### Data preprocessing

Sample Text cleaning before fitting into model is applied here. 
- Most NLP corpus needs customized cleaning based on datasets.
- But for the purpose of demonstration, we are only keeps words as topics and removing numbers and other special characters.

In [13]:
#Data Pre-processing step

import nltk
nltk.download('stopwords')
import re
from nltk.corpus import stopwords
stop=stopwords.words('english')
pat1=re.compile(r"[^a-zA-Z ]+")
pat2=re.compile(r'\b(?:{})\b'.format('|'.join(stop)))

#Data cleaning for various datasets
start_time = time.time()
data_0 = pd.DataFrame(data_0)
data_0.columns = ['train']
data_0 = data_0.train.astype(str).str.replace(pat1," ").str.replace(pat2," ").str.strip()
end_time = time.time() - start_time
print("Time to vectorize dataset 1(in s): "+ str(np.round(end_time, decimals=2)))

start_time = time.time()
data_1 = data_1.data.astype(str).str.replace(pat1," ").str.replace(pat2," ").str.strip()
end_time = time.time() - start_time
print("Time to vectorize dataset 2(in s): "+ str(np.round(end_time, decimals=2)))

start_time = time.time()
data_2 = data_2[0].astype(str).str.replace(pat1," ").str.replace(pat2," ").str.strip()
end_time = time.time() - start_time
print("Time to vectorize dataset 3(in s): "+ str(np.round(end_time, decimals=2)))

start_time = time.time()
data_3 = data_3.train.astype(str).str.replace(pat1," ").str.replace(pat2," ").str.strip()
end_time = time.time() - start_time
print("Time to vectorize dataset 4(in s): "+ str(np.round(end_time, decimals=2)))

# def random_function(x):
#     return ((str(x).replace(r"[^a-zA-Z ]+", " ")).strip())
# data_0 = np.vectorize(random_function)(data_0.data) 
# data_1 = np.vectorize(random_function)(data_1[0].head(20000))
# data_3=data_3.head(10000)
# data_0 = data_0.data.apply(str).str.replace(r"[^a-zA-Z ]+", " ").str.strip()
# data_3=data_3['train'].astype(str).str.replace(r"[^a-zA-Z ]+", " ").str.strip()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Time to vectorize dataset 1(in s): 4.54
Time to vectorize dataset 2(in s): 4.97
Time to vectorize dataset 3(in s): 0.54


NameError: name 'data_3' is not defined

In [14]:
#Storing pre-procssed data into a folder to prevent the need of data pre-processing in future iterations
processed_data_folder = "./processed_data/"
for i in range(len(bucket_list)):
    print("Dataset"+str(i)+" info: ")
    globals()["data_" + str(i)] = globals()["data_" + str(i)].dropna()
    # print(globals()["data_" + str(i)].info())
    print("Size: "+str(sys.getsizeof(globals()["data_" + str(i)])))
    print("Memory: "+str((globals()["data_" + str(i)]).memory_usage(index=True)))
    print()
    print("---------")
    globals()["data_" + str(i)].to_csv(processed_data_folder+"data_"+str(i)+".csv", header=["data"],index=False)

Dataset0 info: 
Size: 30896294
Memory: 301536

---------
Dataset1 info: 
Size: 34454328
Memory: 8160

---------
Dataset2 info: 
Size: 15418194
Memory: 3634288

---------


### Data Reloading

This step is helpful when data loading and preprocessing has happened once, and we do not want to relaod the data again.
- There are some data cleaning steps like removal of text which only consist of numbers or duplicate text, but it should be customized based on dataset
- You can avoid this step during the first run as all the dataset variables point to right dataset

In [80]:
# Reading data from the processed folder so that we can skip data load and preprocessing in future runs
read_from_processed = False   #Change it to True if you want to load data from processed folder
bucket_list = ['docs', './data/CUADv1.json', './data/an4_sphere.tar.gz', 'data_wiki']
if read_from_processed==True:
    for i in range(len(bucket_list)):
        globals()["data_" + str(i)] = pd.read_csv("processed_data/data_"+str(i)+".csv", dtype=str,index_col=None)
        # We can do some additional cleaning like sentences that only have numbers
        globals()["data_" + str(i)].data = globals()["data_" + str(i)].apply(lambda r: r['data'] if type(r['data'])!=float else np.nan, axis=1)
        globals()["data_" + str(i)].dropna(inplace=True)
        globals()["data_" + str(i)].drop_duplicates(inplace=True)
        globals()["data_" + str(i)].reset_index(drop=True, inplace=True)
        globals()["data_" + str(i)] = globals()["data_" + str(i)].squeeze()

# Topic Modeling using BERTopic [without RAPIDS integration]

### Training the BERTopic model

For instantiating BERTopic, you need to run the following commands
- topic_model = BERTopic(verbose=True)
- topics, probs = topic_model.fit_transform(docs)

We can customize BERTopic by using the different parameters mentioned in [BERTopic](https://maartengr.github.io/BERTopic/faq.html). You can add parameters like hdbscan_model=HDBSCAN(), umap_model=UMAP(), language="english" or calculate_probabilities=True in BERTopic() to customize the model.

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model. 

In [41]:
visualize = True  #Parameter to decide if you want the visual results of BERTopic or not
for i in range(len(bucket_list)):
    if i>2:   #Counter to run specific datasets
        continue
    globals()["start_time_" + str(i)] = time.time()
    print(time.ctime())
    globals()["topic_model_gpu_" + str(i)] = BERTopic(verbose=True, nr_topics="auto", hdbscan_model=HDBSCAN(), umap_model=UMAP())
    globals()["topics_" + str(i)], globals()["probs_" + str(i)] = globals()["topic_model_gpu_" + str(i)].fit_transform(globals()["data_" + str(i)])
    globals()["end_time_gpu_" + str(i)] = time.time() - globals()["start_time_" + str(i)]
    print("Time to perform topic modeling for dataset" +str(i+1) + "(in s): "+ str(np.round(globals()["end_time_gpu_" + str(i)], decimals=2)))
    
    if visualize == True:
        viz_topics = globals()["topic_model_gpu_" + str(i)].visualize_topics()
        # print(viz_topics.show())
        viz_topics.write_html("results/viz_topics_"+str(i)+".html")
        viz_barchart = globals()["topic_model_gpu_" + str(i)].visualize_barchart()
        # print(viz_barchart.show())
        viz_barchart.write_html("results/viz_barchart_"+str(i)+".html")
        viz_docs = globals()["topic_model_gpu_" + str(i)].visualize_documents(globals()["data_" + str(i)])
        viz_docs.write_html("results/viz_docs_"+str(i)+".html")
# globals()["topic_model_gpu_" + str(i)] = globals()["topic_model_gpu_" + str(i)].fit(globals()["data_" + str(i)])
# globals()["topics_" + str(i)] = globals()["topic_model_gpu_" + str(i)]._map_predictions(globals()["topic_model_gpu_" + str(i)].hdbscan_model.labels_) 
# globals()["probs_" + str(i)] = globals()["topic_model_gpu_" + str(i)].hdbscan_model.probabilities_

Thu Sep  1 05:09:38 2022


Batches: 100%|██████████| 589/589 [00:09<00:00, 59.31it/s] 
2022-09-01 05:09:49,882 - BERTopic - Transformed documents to Embeddings
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2022-09-01 05:10:07,484 - BERTopic - Reduced dimensionality
2022-09-01 05:10:08,615 - BERTopic - Clustered reduced embeddings
2022-09-01 05:10:17,747 - BERTopic - Reduced number of topics from 603 to 127


Time to perform topic modeling for dataset1(in s): 39.69
Thu Sep  1 05:10:44 2022


Batches: 100%|██████████| 16/16 [00:01<00:00, 11.06it/s]
2022-09-01 05:10:48,076 - BERTopic - Transformed documents to Embeddings
2022-09-01 05:10:49,928 - BERTopic - Reduced dimensionality
2022-09-01 05:10:49,949 - BERTopic - Clustered reduced embeddings
2022-09-01 05:10:56,714 - BERTopic - Reduced number of topics from 25 to 17


Time to perform topic modeling for dataset2(in s): 12.02
Thu Sep  1 05:11:03 2022


Batches: 100%|██████████| 7099/7099 [00:28<00:00, 248.77it/s]
2022-09-01 05:11:33,562 - BERTopic - Transformed documents to Embeddings
2022-09-01 05:16:10,446 - BERTopic - Reduced dimensionality
2022-09-01 05:16:24,543 - BERTopic - Clustered reduced embeddings
2022-09-01 05:16:52,058 - BERTopic - Reduced number of topics from 6098 to 138


Time to perform topic modeling for dataset3(in s): 348.39


### Extracting topics

- After fitting the model, we usually look at the most frequent topics first as they best represent the collection of documents.
- -1 should be ignored as it refers to outliers. 

**NOTE**: Due to stochastic nature of UMAP stage of BERTopic, the topics might differ across runs. 

In [19]:
topic_model_gpu_0.get_topic_info()

,Topic,Count,Name
0,-1,7075,-1_edu_com_the_subject
1,0,857,0_baseball_game_year_team
2,1,780,1_key_encryption_clipper_chip
3,2,306,2_gun_guns_firearms_militia
4,3,279,3_car_cars_mustang_engine
...,...,...,...
292,291,10,291_kent_mcs_mhamilto_nimitz
293,292,10,292_motivated_religously_religiously_frank
294,293,10,293_liar_rh_lunatic_gathered
295,294,10,294_clinton_reno_federal_batf


In [20]:
topic_model_gpu_0.get_topic(0)

[('baseball', 0.010475889952278009),
 ('game', 0.00888648349492628),
 ('year', 0.00861105105443462),
 ('team', 0.008469061639788836),
 ('players', 0.007548359382496322),
 ('hit', 0.007530132235814117),
 ('braves', 0.007529716506300251),
 ('games', 0.007309955232745601),
 ('runs', 0.0068805481485816585),
 ('pitching', 0.006444728759187212)]

In [21]:
topic_model_gpu_1.get_topic_info()

,Topic,Count,Name
0,-1,159,-1_agreement_party_shall_contract
1,0,59,0_distributor_agreement_party_ex
2,1,58,1_agreement_party_company_contract
3,2,32,2_sponsorship_sponsor_agreement_contract
4,3,31,3_maintenance_agreement_shall_vendor
5,4,22,4_endorsement_contract_agreement_ex
6,5,21,5_co_verticalnet_branding_party
7,6,20,6_hosting_customer_software_agreement
8,7,19,7_product_shall_agreement_party
9,8,17,8_msl_ibm_customer_outsourcing


In [22]:
topic_model_gpu_1.get_topic(0)

[('distributor', 0.037079294786719036),
 ('agreement', 0.03270249443242861),
 ('party', 0.030775109377613134),
 ('ex', 0.026850783858802674),
 ('contract', 0.026406133174178816),
 ('shall', 0.026399277154165558),
 ('license', 0.022018705365346277),
 ('related', 0.019295638152168407),
 ('licensee', 0.01887526968572377),
 ('parts', 0.01856044081063633)]

# Topic Modeling using RAPIDS[cuML] integrated BERTopic

### Training the RAPIDS integrated BERTopic model

For instantiating RAPIDS integrated BERTopic, you need to run the following commands
- topic_model = BERTopic(verbose=True, hdbscan_model=HDBSCAN_gpu(), umap_model=UMAP_gpu())
- topics, probs = topic_model.fit_transform(docs)

As we see above, RAPIDS is integrated with BERTopic in such a way that we can use RAPIDS cuML library with couple of changes in BERTopic() parameters.

We can customize BERTopic by using the different parameters mentioned in [BERTopic](https://maartengr.github.io/BERTopic/faq.html#can-i-use-the-gpu-to-speed-up-the-model). 

In [46]:
from bertopic import BERTopic
from cuml.cluster import HDBSCAN as HDBSCAN_gpu
from cuml.manifold import UMAP as UMAP_gpu
from cuml.preprocessing import normalize

In [221]:
visualize = True  #Set to True if you want to visualize the topics 
for i in range(len(bucket_list)):
    if i>2:     #Counter to run specific datasets
        continue
    globals()["start_time_" + str(i)] = time.time()
    print(time.ctime())
    globals()["umap_model_" + str(i)] = UMAP_gpu(n_components=5, n_neighbors=15, min_dist=0.0)
    globals()["hdbscan_model_" + str(i)] = HDBSCAN_gpu(min_samples=10, gen_min_span_tree=True)
    # Pass the above models to be used in BERTopic
    globals()["topic_model_cubert_" + str(i)] = BERTopic(verbose=True, nr_topics="auto",umap_model=globals()["umap_model_" + str(i)], hdbscan_model=globals()["hdbscan_model_" + str(i)])
    globals()["topics_" + str(i)], globals()["probs_" + str(i)] = globals()["topic_model_cubert_" + str(i)].fit_transform(globals()["data_" + str(i)])
    globals()["end_time_cubert_" + str(i)] = time.time() - globals()["start_time_" + str(i)]
    print("Time to perform topic modeling for dataset" +str(i+1) + "(in s): "+ str(np.round(globals()["end_time_cubert_" + str(i)], decimals=2)))
    
    if visualize == True:
        viz_topics = globals()["topic_model_cubert_" + str(i)].visualize_topics()
        # print(viz_topics.show())
        viz_topics.write_html("results/viz_topics_cu_"+str(i)+".html")
        viz_barchart = globals()["topic_model_cubert_" + str(i)].visualize_barchart()
        # print(viz_barchart.show())
        viz_barchart.write_html("results/viz_barchart_cu_"+str(i)+".html")
        viz_docs = globals()["topic_model_cubert_" + str(i)].visualize_documents(globals()["data_" + str(i)])
        viz_docs.write_html("results/viz_docs_cu_"+str(i)+".html")

Mon Aug 15 01:23:00 2022


Batches:   0%|          | 0/589 [00:00<?, ?it/s]

2022-08-15 01:23:14,864 - BERTopic - Transformed documents to Embeddings
2022-08-15 01:23:15,055 - BERTopic - Reduced dimensionality
2022-08-15 01:23:15,426 - BERTopic - Clustered reduced embeddings
2022-08-15 01:23:25,731 - BERTopic - Reduced number of topics from 131 to 85


Time to perform topic modeling for dataset1(in s): 25.07
Mon Aug 15 01:23:52 2022


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2022-08-15 01:23:57,916 - BERTopic - Transformed documents to Embeddings
2022-08-15 01:23:57,947 - BERTopic - Reduced dimensionality
2022-08-15 01:23:58,098 - BERTopic - Clustered reduced embeddings
2022-08-15 01:24:06,724 - BERTopic - Reduced number of topics from 12 to 12


Time to perform topic modeling for dataset2(in s): 14.05
Mon Aug 15 01:24:14 2022


Batches:   0%|          | 0/7099 [00:00<?, ?it/s]

2022-08-15 01:24:50,108 - BERTopic - Transformed documents to Embeddings
2022-08-15 01:25:09,170 - BERTopic - Reduced dimensionality
2022-08-15 01:25:16,469 - BERTopic - Clustered reduced embeddings
2022-08-15 01:25:18,804 - BERTopic - Reduced number of topics from 830 to 99


Time to perform topic modeling for dataset3(in s): 64.33


In [47]:
# Results of another run

Batches:   0%|          | 0/589 [00:00<?, ?it/s]

2022-08-14 01:22:23,941 - BERTopic - Transformed documents to Embeddings
2022-08-14 01:22:25,263 - BERTopic - Reduced dimensionality
2022-08-14 01:22:25,617 - BERTopic - Clustered reduced embeddings
2022-08-14 01:22:34,626 - BERTopic - Reduced number of topics from 135 to 21


Time to perform topic modeling for dataset1(in s): 27.11


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2022-08-14 01:22:38,841 - BERTopic - Transformed documents to Embeddings
2022-08-14 01:22:38,867 - BERTopic - Reduced dimensionality
2022-08-14 01:22:39,008 - BERTopic - Clustered reduced embeddings
2022-08-14 01:22:46,968 - BERTopic - Reduced number of topics from 16 to 16


Time to perform topic modeling for dataset2(in s): 12.34


Batches:   0%|          | 0/7099 [00:00<?, ?it/s]

2022-08-14 01:23:25,953 - BERTopic - Transformed documents to Embeddings
2022-08-14 01:23:45,093 - BERTopic - Reduced dimensionality
2022-08-14 01:23:52,400 - BERTopic - Clustered reduced embeddings
2022-08-14 01:23:54,736 - BERTopic - Reduced number of topics from 830 to 105


Time to perform topic modeling for dataset3(in s): 67.77


In [175]:
#Due to huge size of wikipedia data, it was run separately
visualize = True
for i in range(len(bucket_list)):
    if i<=2:     #Counter to run specific datasets
        continue
    globals()["start_time_" + str(i)] = time.time()
    print(time.ctime())
    globals()["umap_model_" + str(i)] = UMAP_gpu(n_components=5, n_neighbors=15, min_dist=0.0)
    globals()["hdbscan_model_" + str(i)] = HDBSCAN_gpu(min_samples=10, gen_min_span_tree=True)
    # Pass the above models to be used in BERTopic
    globals()["topic_model_cubert_" + str(i)] = BERTopic(verbose=True, nr_topics="auto",umap_model=globals()["umap_model_" + str(i)], hdbscan_model=globals()["hdbscan_model_" + str(i)])
    globals()["topics_" + str(i)], globals()["probs_" + str(i)] = globals()["topic_model_cubert_" + str(i)].fit_transform(globals()["data_" + str(i)])
    globals()["end_time_cubert_" + str(i)] = time.time() - globals()["start_time_" + str(i)]
    print("Time to perform topic modeling for dataset" +str(i+1) + "(in s): "+ str(np.round(globals()["end_time_cubert_" + str(i)], decimals=2)))
    
    if visualize == True:
        viz_topics = globals()["topic_model_cubert_" + str(i)].visualize_topics()
        # print(viz_topics.show())
        viz_topics.write_html("results/viz_topics_cu_"+str(i)+".html")
        viz_barchart = globals()["topic_model_cubert_" + str(i)].visualize_barchart()
        # print(viz_barchart.show())
        viz_barchart.write_html("results/viz_barchart_cu_"+str(i)+".html")
        viz_docs = globals()["topic_model_cubert_" + str(i)].visualize_documents(globals()["data_" + str(i)])
        viz_docs.write_html("results/viz_docs_cu_"+str(i)+".html")

2022-08-14 07:28:30,891 - BERTopic - Transformed documents to Embeddings
2022-08-14 07:29:42,060 - BERTopic - Reduced dimensionality
2022-08-14 07:29:58,296 - BERTopic - Clustered reduced embeddings


Time to perform topic modeling for dataset4(in s): 1334.17


### Extracting topics

- After fitting the model, we usually look at the most frequent topics first as they best represent the collection of documents.
- -1 should be ignored as it refers to outliers. 

**NOTE**: Due to stochastic nature of UMAP stage of BERTopic, the topics might differ across runs. 

In [20]:
topic_model_cubert_0.get_topic_info()

,Topic,Count,Name
0,-1,7967,-1_edu_the_com_subject
1,0,1855,0_game_team_hockey_games
2,1,1807,1_car_bike_dod_com
3,2,818,2_medical_disease_cancer_msg
4,3,692,3_space_nasa_launch_orbit
...,...,...,...
132,131,5,131_joystick_int_port_eyal
133,132,5,132_synoptics_klee_widgets_widget
134,133,5,133_baptists_trincoll_handheld_banging
135,134,5,134_typing_rsi_keyboard_berkeley


In [21]:
topic_model_cubert_0.get_topic(0)

[('game', 0.014723341857636218),
 ('team', 0.014033122834136701),
 ('hockey', 0.010271081247856223),
 ('games', 0.009663337017870146),
 ('players', 0.009230325117226821),
 ('year', 0.009042694970761444),
 ('season', 0.008809628956061221),
 ('play', 0.007999885530012555),
 ('baseball', 0.00795392384225527),
 ('edu', 0.007310806833653132)]

In [22]:
topic_model_cubert_1.get_topic_info()

,Topic,Count,Name
0,0,481,0_agreement_shall_party_contract
1,1,16,1_spinco_intellectual_group_property
2,2,13,2_reseller_agreement_shall_party


In [23]:
topic_model_cubert_1.get_topic(0)

[('agreement', 0.061061634547482896),
 ('shall', 0.051238800336885),
 ('party', 0.04987079300098038),
 ('contract', 0.04087231522561693),
 ('ex', 0.030811017910416637),
 ('related', 0.027721035114121623),
 ('parts', 0.025278805981154232),
 ('details', 0.02523013458650461),
 ('question', 0.024984074548969094),
 ('reviewed', 0.024980114554129226)]

# Results

## Overall Results

In [169]:
#Overall time is calculated for only the topic modeling piece

In [ ]:
results=pd.DataFrame()

results['dataset'] = ['news_dataset', 'CUAD_dataset','AN4_dataset']
results['BERTopic_time (in s)'] = [np.round(end_time_gpu_0,decimals=2),np.round(end_time_gpu_1,decimals=2),np.round(end_time_gpu_2,decimals=2)]
results['BERTopic_time_with_RAPIDS (in s)'] = [np.round(end_time_cubert_0,decimals=2), np.round(end_time_cubert_1,decimals=2),np.round(end_time_cubert_2,decimals=2)]
results

In [ ]:
# Results
#         dataset       |    BERTopic_time (in s)  |     BERTopic_time [with RAPIDS] (in s)
#    --------------     |   --------------------   |     --------------------------------
# 0 |   news_dataset    |        37.59             |              27.11
# 1 |   CUAD_dataset    |        15.86             |              12.34
# 2 |   AN4_dataset     |        1823.34           |              67.77

In [214]:
results.to_csv('./results/overall_results.csv', index = False)

## Deep dive into datasets - Performances of BERTopic with and without RAPIDS integration are gauged in various stages of Topic modeling

### BERTopic Stages [without RAPIDS integration]

#### Topic Modeling of AN4 Dataset

In [34]:
topic_bert_model = BERTopic(verbose=True, nr_topics="auto")
data=data_2
docs = pd.DataFrame({"Document": data, "ID":range(len(data)), "Topic":None})

In [35]:
%%time
#Extract embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings=model.encode(data, batch_size=64, show_progress_bar=True
                        # ,#device='cpu'
                       )

Batches:   0%|          | 0/3550 [00:00<?, ?it/s]

CPU times: user 2min 46s, sys: 15.7 s, total: 3min 2s
Wall time: 22 s


In [36]:
%%time
#Dimensionality Reduction using UMAP
umap_embeddings = topic_bert_model._reduce_dimensionality(embeddings)

2022-08-13 21:47:00,201 - BERTopic - Reduced dimensionality


CPU times: user 2h 56min 15s, sys: 16.7 s, total: 2h 56min 32s
Wall time: 27min 40s


In [37]:
%%time
#Cluster UMAP embeddings with HDBSCAN
docs, probs = topic_bert_model._cluster_embeddings(umap_embeddings, docs)

2022-08-13 21:49:37,330 - BERTopic - Clustered reduced embeddings


CPU times: user 52.8 s, sys: 2.16 s, total: 55 s
Wall time: 57.8 s


In [38]:
%%time
#Sort and map Topic IDs by frequency
if not topic_bert_model.nr_topics:
    docs = topic_bert_model._sort_mappings_by_frequency(docs)

#Extract topics by calculating c-TF-IDF
topic_bert_model._extract_topics(docs) #For topic extraction and representation

CPU times: user 417 ms, sys: 32 ms, total: 449 ms
Wall time: 448 ms


#### Topic Modeling of Wikidata dataset

In [203]:
topic_bert_model = BERTopic(verbose=True, nr_topics="auto")
data=data_3
docs = pd.DataFrame({"Document": data, "ID":range(len(data)), "Topic":None})

In [204]:
%%time
#Extract embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings=model.encode(data, batch_size=64, show_progress_bar=True)

Batches:   0%|          | 0/7813 [00:00<?, ?it/s]

CPU times: user 1h 53min 29s, sys: 1min 7s, total: 1h 54min 37s
Wall time: 11min 23s


In [205]:
%%time
#Dimensionality Reduction using UMAP
umap_embeddings = topic_bert_model._reduce_dimensionality(embeddings)

2022-08-14 23:14:45,145 - BERTopic - Reduced dimensionality


CPU times: user 2h 57min 5s, sys: 1min 18s, total: 2h 58min 24s
Wall time: 13min 10s


In [209]:
%%time
#Cluster UMAP embeddings with HDBSCAN
docs, probs = topic_bert_model._cluster_embeddings(umap_embeddings, docs)

2022-08-15 00:18:02,205 - BERTopic - Clustered reduced embeddings


CPU times: user 54.1 s, sys: 5.36 s, total: 59.5 s
Wall time: 1min 5s


In [210]:
%%time
#Sort and map Topic IDs by frequency
if not topic_bert_model.nr_topics:
    docs = topic_bert_model._sort_mappings_by_frequency(docs)

#Extract topics by calculating c-TF-IDF
topic_bert_model._extract_topics(docs) #For topic extraction and representation

CPU times: user 8min 16s, sys: 8.47 s, total: 8min 24s
Wall time: 8min 24s


## cuBERTopic Stages

cuBERT is a standalone package that is built to optimize UMAP phase of BERTopic 
- To gauge the performance of various stages of cuBERTopic, we need to fork the git repository from [RAPIDS git repo](https://github.com/rapidsai/rapids-examples/tree/main/cuBERT_topic_modeling) 

In [48]:
import cudf
import cupy

#### Topic Modeling of AN4 Dataset

In [154]:
#Change the below path to reflect location where you cloned the RAPIDS github repo
cu_bert_path = '../rapids-examples/cuBERT_topic_modelling/'
os.chdir(cu_bert_path)
from cuBERTopic import gpu_BERTopic
from embedding_extraction import create_embeddings

In [155]:
topic_cubert_model = gpu_BERTopic()
data = "data_2"
test_dataset = cudf.read_csv('../../NVIDIA-Devrel/processed_data/'+ data +'.csv')
test_dataset.columns

Index(['Unnamed: 0', '0'], dtype='object')

In [156]:
data = test_dataset['0']
docs = cudf.DataFrame({"Document": data, "ID":cp.arange(len(data)), "Topic":None})
embedding_model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2").to("cuda")

In [157]:
%%time
#Extract embeddings
embeddings = create_embeddings(docs.Document, embedding_model, vocab_file='../cuBERT_topic_modelling/vocab/voc_hash.txt')
# model.encode(data, batch_size=64, show_progress_bar=True)

CPU times: user 2min 11s, sys: 39.1 ms, total: 2min 11s
Wall time: 2min 11s


In [158]:
%%time
#Dimensionality Reduction using UMAP
umap_embeddings = topic_cubert_model.reduce_dimensionality(embeddings)

CPU times: user 7.04 s, sys: 112 ms, total: 7.15 s
Wall time: 7.15 s


In [159]:
%%time
#Cluster UMAP embeddings with HDBSCAN
docs_cubert_model, probs_cubert_model = topic_cubert_model.cluster_embeddings(umap_embeddings, docs)

CPU times: user 6.6 s, sys: 128 ms, total: 6.73 s
Wall time: 6.72 s


In [162]:
# %%time
# # c-TF-IDF 
# tf_idf, vectorizer, topic_labels = topic_cubert_model.create_topics(docs_cubert_model)

In [161]:
# #Topic representation
# top_n_words, name_repr = topic_cubert_model.extract_top_n_words_per_topic(tf_idf, vectorizer, topic_labels,n=30)

# topic_cubert_model.topic_sizes_df["Name"] = topic_cubert_model.topic_sizes_df['Topic'].map(name_repr)

In [201]:
os.chdir('../../NVIDIA-Devrel/')

# Appendix

## Running customized BERTopic [without RAPIDS integration]

In [26]:
%%time
# Customizing HDBSCAN 
hdbscan_model_1 = HDBSCAN(min_cluster_size=10, metric='euclidean', 
                        cluster_selection_method='eom', prediction_data=True, min_samples=5)
topic_model_1 = BERTopic(hdbscan_model=hdbscan_model_1)
topics_gpu_1, probs_gpu_1 = topic_model_1.fit_transform(data_0)

CPU times: user 8min 7s, sys: 10.6 s, total: 8min 18s
Wall time: 47.6 s


In [27]:
#Customizng embedding
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

In [28]:
%%time
# Customizing embedding 
embeddings = sentence_model.encode(data_0, show_progress_bar=False)
topic_model_2 = BERTopic()
topics_gpu_2, probs_gpu_2 = topic_model_2.fit_transform(data_0, embeddings)

CPU times: user 10min 5s, sys: 5.58 s, total: 10min 10s
Wall time: 31.2 s


## Running customized RAPIDs integrated BERTopic

In [30]:
from bertopic import BERTopic
from cuml.cluster import HDBSCAN as HDBSCAN_gpu
from cuml.manifold import UMAP as UMAP_gpu
from cuml.preprocessing import normalize

In [31]:
%%time
# Create instances of GPU-accelerated UMAP and HDBSCAN
umap_model_3 = UMAP_gpu(n_components=5, n_neighbors=15, min_dist=0.0)
hdbscan_model_3 = HDBSCAN_gpu(min_samples=10, gen_min_span_tree=True)

# Pass the above models to be used in BERTopic
topic_model_3 = BERTopic(umap_model=umap_model_3, hdbscan_model=hdbscan_model_3)
topics_gpu_3, probs_gpu_3 = topic_model_3.fit_transform(data_0)

CPU times: user 1min 22s, sys: 2.98 s, total: 1min 25s
Wall time: 18.4 s


In [32]:
#Customizng embedding
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

In [33]:
%%time
# Customizing embedding 
embeddings = sentence_model.encode(data_0, show_progress_bar=False)
embeddings = normalize(embeddings)

# Create instances of GPU-accelerated UMAP and HDBSCAN
umap_model_4 = UMAP_gpu(n_components=5, n_neighbors=15, min_dist=0.0)
hdbscan_model_4 = HDBSCAN_gpu(min_samples=10, gen_min_span_tree=True)

# Pass the above models to be used in BERTopic
topic_model_4 = BERTopic(umap_model=umap_model_4, hdbscan_model=hdbscan_model_4)
topics_gpu_4, probs_gpu_4 = topic_model_4.fit_transform(data_0, embeddings)

CPU times: user 1min 16s, sys: 2.55 s, total: 1min 19s
Wall time: 19.5 s
